In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
import numpy as np
import pandas as pd
import plotly.express as px
from investment import (
    Investment,
    ETFInvestment,
    PropertyInvestment
)

In [3]:
def run_simulation(investments: list[Investment], years: int):
    print(f"{'Year':<5} | {'Asset':<15} | {'Value':<15}")
    print("-" * 40)
    
    for year in range(1, years + 1):
        for asset in investments:
            # The Polymorphism magic: 
            # The engine doesn't care if it's Property or ETF, 
            # it just calls the standard methods.
            asset.calculate_annual_return()
            asset.apply_costs()
            asset.log_status(year)
            
            if year % 5 == 0: # Print every 5th year
                print(f"{year:<5} | {asset.name:<15} | ${asset.value:,.2f}")

# Running it
my_etf = ETFInvestment("Vanguard S&P", 100000, 0.08, 0.003)
my_prop = PropertyInvestment("Downtown Condo", 500000, 0.05, 0.04, 2000)

run_simulation([my_etf, my_prop], 20)

Year  | Asset           | Value          
----------------------------------------
5     | Vanguard S&P    | $144,742.00
5     | Downtown Condo  | $638,140.78
10    | Vanguard S&P    | $209,502.47
10    | Downtown Condo  | $814,447.31
15    | Vanguard S&P    | $303,238.06
15    | Downtown Condo  | $1,039,464.09
20    | Vanguard S&P    | $438,912.83
20    | Downtown Condo  | $1,326,648.85
